In [1]:
import os
os.chdir("../")

In [2]:
# Credential in the environment
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/M-Hamza380/Kidney-Disease-DL-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "M-Hamza380"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "cf3e0fd7ca33059236f97d51ff962b7dd1292f89"

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
# Define entity
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from src.classifier.constants import *
from src.classifier.utils import read_yaml, create_directories, save_json

In [8]:
# Define Configuration Manager
class ConfigurationManager:
    def __init__(self,
                 config_filepath= Config_File_Path,
                 params_filepath= Params_File_Path
                 ):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_evaluate_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/Kidney-CT-Scan-data',
            mlflow_uri= 'https://dagshub.com/M-Hamza380/Kidney-Disease-DL-Project.mlflow',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_config

In [9]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
# Define Components

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict (
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory =self.config.training_data,
            subset= 'validation',
            shuffle= False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path= Path('score.json'), data= scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {'loss': self.score[0], 'accuracy': self.score[1]}
            )
            # Model registry does not work with file
            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(self.model, 'model', registered_model_name= 'VGG16Model')
            else:
                mlflow.keras.log_model(self.model, 'model')

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluate_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 804 images belonging to 2 classes.
51/51 [==============================] - 322s 6s/step - loss: 1.3890 - accuracy: 0.8060


2024/02/11 22:06:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\hamza\AppData\Local\Temp\tmpgoj6f6xd\model\data\model\assets


f:\Git_Projects\Kidney-Disease-DL-Project\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/02/11 22:09:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
